In [25]:
import pandas as pd
import numpy as np
import pymssql
from yaml import Loader

In [27]:
with open('secrets.yaml', 'r') as f:
    configs = yaml.load(f, Loader=Loader)

In [30]:

server = configs['data']['server']
user = configs['data']['user']
password = configs['data']['password']
database = configs['data']['database']

# define table strings
efficiency_table = 'dbo.EfficiencyScores'
safety_table = 'dbo.SafetyScores'
outcomes_table = 'dbo.ClinicalOutcomeScores'
community_table = 'dbo.EngagementScores'
payment_table = 'PaymentAndValueOfCareVals'

try:
    # connect to database with above credentials
    conn = pymssql.connect(server, user, password, database)
    
    # instantiate cursor
    cursor = conn.cursor()
    
    # get efficiency data
    efficiency_query = f'SELECT * FROM {efficiency_table}'
    efficiency = pd.read_sql(efficiency_query, conn, index_col='Efficiency_ID')
    
    # get safety data
    safety_query = f'SELECT * FROM {safety_table}'
    safety = pd.read_sql(safety_query, conn, index_col='Safety_ID')
    
    # get outcomes data
    outcomes_query = f'SELECT * FROM {outcomes_table}'
    outcomes = pd.read_sql(outcomes_query, conn, index_col='ClinicalOutcome_ID')
    
    # get community data
    community_query = f'SELECT * FROM {community_table}'
    community = pd.read_sql(community_query, conn, 'EngagementScore_ID')
    
    #get payment data
    payment_query = f'SELECT * FROM {payment_table}'
    payment = pd.read_sql(payment_query, conn, index_col='Payment_ID')
except Exception as e:
    print(e)
    
payment.head()

/home/charlie/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/charlie/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/charlie/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/charlie/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/conne

,Facility_ID,Payment,Lower_Estimate,Higher_Estimate,Payment_Category,Value_Of_Care_Category
Payment_ID,,,,,,
1,020018,13461.0000,11689.5000,15405.5000,-0.5,-0.5
2,030064,20773.2500,18978.2500,22704.0000,0.25,0
3,100140,18872.6667,17511.3333,20323.3333,-0.333333,0
4,100320,18230.0000,16388.5000,20246.5000,0,0
5,110071,18356.0000,15727.0000,20892.0000,0,0


In [43]:
joined = efficiency.merge(safety, on='Facility_ID', how='inner')
joined_1  = joined.merge(outcomes, on='Facility_ID', how='inner')
joined_2 = joined.merge(community, on='Facility_ID', how='inner')
final_join = payment.merge(joined_2, on='Facility_ID', how='left')

In [46]:
final_join.head()

,Facility_ID,Payment,Lower_Estimate,Higher_Estimate,Payment_Category,Value_Of_Care_Category,MSPB_Baseline,MSPB_Performance,HAI_Baseline,HAI_Performance,...,Care_Transition_Baseline,Care_Transition_Perform,Comm_Medicines_Baseline,Comm_Medicines_Perform,Clean_Quiet_Baseline,Clean_Quiet_Perform,Discharge_Info_Baseline,Discharge_Info_Perform,Overall_Rating_Baseline,Overall_Rating_Perform
0,020018,13461.0000,11689.5000,15405.5000,-0.5,-0.5,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,030064,20773.2500,18978.2500,22704.0000,0.25,0,0.968294,0.975835,0.617,0.733,...,50.928200,43.850899,64.486801,58.995300,53.257900,58.215302,85.283798,85.326103,65.936401,67.765297
2,100140,18872.6667,17511.3333,20323.3333,-0.333333,0,0.963651,0.959826,0.153,0.313,...,54.586800,53.125000,67.964401,59.971298,73.614899,75.125397,88.297501,87.005402,78.330299,78.791901
3,100320,18230.0000,16388.5000,20246.5000,0,0,0.961516,0.896499,0.553,0.238,...,46.411098,44.534000,60.029099,54.170502,69.264999,73.066200,79.250000,79.810799,64.692001,63.960098
4,110071,18356.0000,15727.0000,20892.0000,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
